# Idée d'application 
- Prédiction du nutriscore à partir des macronutriments qui composent un aliment.
    
# Nettoyage des données

* [Sélection de variables](#Sélection-de-variables)
* [Sélection d'observations](#Sélection-d'observations)
* [Création de variables](#Création-de-variables)
* [Elimination des sources d'erreurs](#Elimination-des-sources-d'erreurs)
* [Imputation des valeurs manquantes](#Imputation-des-valeurs-manquantes)

In [1]:
import pandas as pd
import numpy as np
from missingpy import MissForest
import time
pd.set_option('display.max_columns', None)

C:\Users\Quentin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv("en.openfoodfacts.org.products.csv", sep="\t", encoding="utf-8")
df.head(3)

C:\Users\Quentin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,8,13,27,28,29,31,47,52,55,64) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
df.shape

(1964849, 186)

In [4]:
def get_missing_values_table(d) :
        mis_val = d.isnull().sum()
        mis_val_percent = 100 * d.isnull().sum() / len(d)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending = False).round(2)
        print (f">>> df contient {mis_val_table_ren_columns.shape[0]} colonnes avec des valeurs manquantes.")
        return mis_val_table_ren_columns.T

In [5]:
get_missing_values_table(df)

>>> df contient 177 colonnes avec des valeurs manquantes.


,allergens_en,no_nutriments,ingredients_that_may_be_from_palm_oil,cities,ingredients_from_palm_oil,water-hardness_100g,additives,-elaidic-acid_100g,glycemic-index_100g,chlorophyl_100g,-caproic-acid_100g,nutrition-score-uk_100g,-caprylic-acid_100g,-erucic-acid_100g,-dihomo-gamma-linolenic-acid_100g,-capric-acid_100g,-nervonic-acid_100g,-gamma-linolenic-acid_100g,-stearic-acid_100g,-cerotic-acid_100g,-mead-acid_100g,-myristic-acid_100g,-lignoceric-acid_100g,-montanic-acid_100g,-melissic-acid_100g,-lauric-acid_100g,-butyric-acid_100g,beta-glucan_100g,carnitine_100g,-behenic-acid_100g,-palmitic-acid_100g,nucleotides_100g,-gondoic-acid_100g,casein_100g,choline_100g,inositol_100g,serum-proteins_100g,-oleic-acid_100g,-maltose_100g,-fructose_100g,beta-carotene_100g,-glucose_100g,omega-9-fat_100g,-arachidic-acid_100g,-eicosapentaenoic-acid_100g,-arachidonic-acid_100g,silica_100g,-sucrose_100g,ph_100g,-maltodextrins_100g,chromium_100g,taurine_100g,-docosahexaenoic-acid_100g,collagen-meat-protein-ratio_100g,molybdenum_100g,fluoride_100g,bicarbonate_100g,-linoleic-acid_100g,carbon-footprint_100g,caffeine_100g,starch_100g,omega-6-fat_100g,fruits-vegetables-nuts-dried_100g,-alpha-linolenic-acid_100g,chloride_100g,-lactose_100g,energy-from-fat_100g,biotin_100g,vitamin-k_100g,phylloquinone_100g,omega-3-fat_100g,iodine_100g,selenium_100g,-insoluble-fiber_100g,-soluble-fiber_100g,vitamin-e_100g,polyols_100g,manganese_100g,copper_100g,cocoa_100g,pantothenic-acid_100g,packaging_text,abbreviated_product_name,folates_100g,fruits-vegetables-nuts_100g,vitamin-d_100g,zinc_100g,vitamin-b9_100g,carbon-footprint-from-meat-or-fish_100g,fruits-vegetables-nuts-estimate_100g,vitamin-b12_100g,phosphorus_100g,ingredients_from_palm_oil_tags,magnesium_100g,vitamin-b6_100g,alcohol_100g,vitamin-b2_100g,vitamin-b1_100g,vitamin-pp_100g,ingredients_that_may_be_from_palm_oil_tags,polyunsaturated-fat_100g,monounsaturated-fat_100g,first_packaging_code_geo,cities_tags,origins_tags,origins_en,origins,potassium_100g,traces,emb_codes_tags,generic_name,emb_codes,manufacturing_places_tags,manufacturing_places,traces_tags,traces_en,energy-kj_100g,purchase_places,allergens,vitamin-a_100g,vitamin-c_100g,stores,trans-fat_100g,iron_100g,cholesterol_100g,calcium_100g,brand_owner,packaging_tags,packaging,additives_tags,additives_en,labels_en,labels_tags,labels,ecoscore_score_fr,ecoscore_grade_fr,quantity,fiber_100g,serving_quantity,serving_size,nova_group,image_ingredients_url,image_ingredients_small_url,nutriscore_score,nutriscore_grade,nutrition-score-fr_100g,ingredients_text,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,categories_en,categories_tags,main_category,main_category_en,categories,image_nutrition_url,image_nutrition_small_url,brands_tags,brands,sodium_100g,salt_100g,image_url,image_small_url,energy-kcal_100g,saturated-fat_100g,sugars_100g,carbohydrates_100g,fat_100g,proteins_100g,energy_100g,product_name,countries_tags,countries_en,countries,pnns_groups_1,pnns_groups_2,creator
Missing Values,1964849.0,1964849.0,1964849.0,1964849.0,1964849.0,1964848.0,1964848.0,1964847.0,1964845.0,1964844.0,1964842.0,1964841.0,1964841.0,1964840.0,1964836.0,1964836.0,1964833.0,1964832.0,1964831.0,1964831.0,1964831.0,1964828.0,1964828.0,1964827.0,1964826.0,1964824.0,1964815.0,1964811.0,1964810.0,1964807.0,1964805.0,1964803.0,1964801.0,1964792.0,1964786.0,1964780.0,1964778.0,1964776.0,1964769.0,1964769.0,1964767.0,1964748.00,1964738.00,1964732.00,1964725.00,1964717.00,1964689.00,1964679.00,1964647.00,1964644.00,1964637.00,1964634.00,1964625.00,1964534.00,1964526.00,1964454.00,1964422.00,1964364.00,1964364.00,1964362.00,1964313.00,1964295.00,1964195.00,1964079.00,1964042.00,1963969.00,1963878.00,1963643.00,1963614.00,1963137.00,1962780.00,1962528.00,1962287.00,1961510.00,1961239.00,1961228.00,1961150.00,1960738.00,1960514.00,1958649.00,1958636.00,1957918.00,1957759.00,1956363.00,1955745.00,1954872.00,1954544.00,1954466.00,1953015.0,1952998.0,1951890.00,19508

# Sélection de variables

In [6]:
nutrition_table_cols = ["product_name", "countries_en", "packaging",
                        "nutriscore_grade", "nutriscore_score", "additives_n",
                        "energy_100g", "fat_100g", "carbohydrates_100g",
                        "sugars_100g", "proteins_100g", "salt_100g"] 

nutrition_table = df[nutrition_table_cols].copy()
print(nutrition_table.shape)
nutrition_table.head(3)

(1964849, 12)


,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,energy_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g
0,jeunes pousses,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L.casei,Spain,NaN,NaN,NaN,0.0,NaN,1.4,9.8,9.8,2.7,0.1
2,Vitória crackers,France,NaN,NaN,NaN,NaN,1569.0,7.0,70.1,15.0,7.8,1.4


In [7]:
get_missing_values_table(nutrition_table)

>>> df contient 12 colonnes avec des valeurs manquantes.


,packaging,nutriscore_grade,nutriscore_score,additives_n,salt_100g,sugars_100g,carbohydrates_100g,fat_100g,proteins_100g,energy_100g,product_name,countries_en
Missing Values,1658068.00,1250059.00,1250059.00,1245626.0,492790.00,429595.00,413942.00,413572.00,412374.00,405630.00,80832.00,6079.00
% of Total Values,84.39,63.62,63.62,63.4,25.08,21.86,21.07,21.05,20.99,20.64,4.11,0.31


# Sélection d'observations

Supprimons les observations ayant :
   - au moins une valeur manquante dans les variables ne correspondant pas à un macronutriment
   - les valeurs manquantes restantes seront comblées via `MissForest()` dans la partie __Imputation des valeurs manquantes__ du notebook.

In [8]:
nutrition_table = nutrition_table.dropna(subset = ["packaging", "nutriscore_grade",
                                                   "nutriscore_score","product_name",
                                                   "countries_en", "additives_n"]).reset_index(drop=True)
print(nutrition_table.shape)
nutrition_table.head(3)

(176776, 12)


,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,energy_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g
0,Blanquette de Volaille et son Riz,France,"carton,plastique",b,0.0,2.0,450.0,2.2,15.3,0.5,6.8,0.70
1,Salade Cesar,Canada,Frais,c,6.0,3.0,1210.0,12.0,23.0,0.0,22.0,2.16
2,Entremets Crème Brulée,France,Sachet,b,2.0,4.0,1952.4,15.3,75.5,69.8,6.8,0.50


In [9]:
get_missing_values_table(nutrition_table) 

>>> df contient 6 colonnes avec des valeurs manquantes.


,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,energy_100g,salt_100g
Missing Values,898.00,776.00,773.00,771.00,769.00,443.00
% of Total Values,0.51,0.44,0.44,0.44,0.44,0.25


# Création de variables

Ajoutons les variables :
- __other_carbs__, contenant la valeur des glucides n'étant pas du sucre. 
- __g_sum__, étant la somme des macronutriments en gramme. 
- __reconstructed_energy_100g__, étant le calcul de l'énergie en kJ à partir des protéines, des lipides et des glucides

![alt text](reconstructed_energy.jpg "Title")

In [10]:
energy_100g_idx = nutrition_table.columns.get_loc("energy_100g")

nutrition_table.insert(energy_100g_idx + 1, "reconstructed_energy_100g",
                       37 * nutrition_table["fat_100g"] +\
                       17 * (nutrition_table["proteins_100g"] + nutrition_table["carbohydrates_100g"]))

sugars_100g_idx = nutrition_table.columns.get_loc("sugars_100g")

nutrition_table.insert(sugars_100g_idx + 1, "other_carbs",
                       nutrition_table["carbohydrates_100g"] - nutrition_table["sugars_100g"])

end_idx = nutrition_table.shape[1]

nutrition_table.insert(end_idx, "g_sum", nutrition_table["fat_100g"] + nutrition_table["carbohydrates_100g"] +\
                                         nutrition_table["proteins_100g"] + nutrition_table["salt_100g"])
print(nutrition_table.shape)
nutrition_table.head(3)

(176776, 15)


,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,energy_100g,reconstructed_energy_100g,fat_100g,carbohydrates_100g,sugars_100g,other_carbs,proteins_100g,salt_100g,g_sum
0,Blanquette de Volaille et son Riz,France,"carton,plastique",b,0.0,2.0,450.0,457.1,2.2,15.3,0.5,14.8,6.8,0.70,25.00
1,Salade Cesar,Canada,Frais,c,6.0,3.0,1210.0,1209.0,12.0,23.0,0.0,23.0,22.0,2.16,59.16
2,Entremets Crème Brulée,France,Sachet,b,2.0,4.0,1952.4,1965.2,15.3,75.5,69.8,5.7,6.8,0.50,98.10


# Elimination des sources d'erreurs

Maintenant que nous avons ajouté quelques variables utiles, nous allons exlure les valeurs aberrantes, c'est-à-dire les produits ayant : 

- une variable __xxx_100g__ supérieure à 100g (à l'exception de __energy_100g__)
- une variable avec une valeur négative
- une entrée pour la variable __energy_100g__ supérieure à 3700kJ (à l'exception de l'huile)
- plus de __sucres__ que de __glucides__

In [11]:
wrong_entries = (nutrition_table["fat_100g"] > 100) |\
                (nutrition_table["carbohydrates_100g"] > 100) |\
                (nutrition_table["proteins_100g"] > 100) |\
                (nutrition_table["salt_100g"] > 100) |\
                (nutrition_table["g_sum"] > 100) |\
                (nutrition_table["fat_100g"] < 0) |\
                (nutrition_table["carbohydrates_100g"] < 0) |\
                (nutrition_table["sugars_100g"] < 0) |\
                (nutrition_table["proteins_100g"] < 0) |\
                (nutrition_table["salt_100g"] < 0) |\
                (nutrition_table["sugars_100g"] > nutrition_table["carbohydrates_100g"]) |\
                ((nutrition_table["energy_100g"] > 3700) & (nutrition_table["fat_100g"] < 100)) |\
                (nutrition_table["energy_100g"] > 10_000)

nutrition_table = nutrition_table[~wrong_entries]
nutrition_table.shape

(175406, 15)

__Nettoyons les variables textuelles :__

 - `product_name` et `countries_name` en minuscule
 - retirer les préfixes tels que **"en:"** de `main_category`
 - `nutriscore_grade` en majuscule

In [12]:
nutrition_table["product_name"] = nutrition_table["product_name"].str.strip().str.lower()
nutrition_table["countries_en"] = nutrition_table["countries_en"].str.lower()
nutrition_table["packaging"] = nutrition_table["packaging"].str.lower()
nutrition_table["nutriscore_grade"] = nutrition_table["nutriscore_grade"].str.capitalize()

- Appliquons la méthode `drop_duplicates()`  à notre dataframe afin de supprimer d'éventuelles doublons exactes 
- Supprimons la variable `energy_100g`, nous utiliserons la variable `reconstructed_energy_100g`

In [13]:
nutrition_table = nutrition_table.drop_duplicates().reset_index(drop=True)
nutrition_table.drop(["energy_100g"], inplace=True, axis=1)
print(nutrition_table.shape)
nutrition_table.head(3)

(174389, 14)


,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,reconstructed_energy_100g,fat_100g,carbohydrates_100g,sugars_100g,other_carbs,proteins_100g,salt_100g,g_sum
0,blanquette de volaille et son riz,france,"carton,plastique",B,0.0,2.0,457.1,2.2,15.3,0.5,14.8,6.8,0.70,25.00
1,salade cesar,canada,frais,C,6.0,3.0,1209.0,12.0,23.0,0.0,23.0,22.0,2.16,59.16
2,entremets crème brulée,france,sachet,B,2.0,4.0,1965.2,15.3,75.5,69.8,5.7,6.8,0.50,98.10


# Imputation des valeurs manquantes

`MissForest()` impute les valeurs manquantes à l'aide de forêts aléatoires de manière itérative. Par défaut, l'imputeur commence à imputer les valeurs manquantes de la colonne (qui devrait être une variable) avec le plus petit nombre de valeurs manquantes -- appelons cela la colonne candidate.

La première étape consiste à remplir toutes les valeurs manquantes des colonnes non candidates restantes avec une estimation initiale, qui est la moyenne de colonne pour les colonnes représentant des variables numériques.

Après cela, l'imputeur ajuste un modèle de forêt aléatoire avec la colonne candidate comme variable de résultat et les colonnes restantes comme prédicteurs sur toutes les lignes où les valeurs de la colonne candidate ne manquent pas.

Après l'ajustement, les lignes manquantes de la colonne candidate sont imputées à l'aide de la prédiction de la forêt aléatoire ajustée. Les lignes des colonnes non candidates servent de données d'entrée pour le modèle ajusté. 

Ensuite, l'imputeur passe à la colonne candidate suivante avec le deuxième plus petit nombre de valeurs manquantes parmi les colonnes non candidates du premier tour. Le processus se répète pour chaque colonne avec une valeur manquante, éventuellement sur plusieurs itérations ou époques pour chaque colonne, jusqu'à ce que le critère d'arrêt soit rempli. 

Le critère d'arrêt est régi par la "différence" entre les tableaux imputés sur des itérations successives. Pour les variables numériques (num_vars_), la différence est définie comme suit :

In [14]:
get_missing_values_table(nutrition_table)

>>> df contient 8 colonnes avec des valeurs manquantes.


,g_sum,other_carbs,reconstructed_energy_100g,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,salt_100g
Missing Values,887.00,872.0,869.0,862.00,742.00,739.00,737.00,423.00
% of Total Values,0.51,0.5,0.5,0.49,0.43,0.42,0.42,0.24


Réalisons l'imputation des valeurs manquantes sur les valeurs correspondant aux macronutriments :

In [16]:
macro_attribs = ["reconstructed_energy_100g", "carbohydrates_100g",
                 "sugars_100g", "fat_100g", "proteins_100g", "salt_100g"]

nutrition_table_to_fill = nutrition_table[macro_attribs]
nutrition_table_to_fill.head(3)

,reconstructed_energy_100g,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,salt_100g
0,457.1,15.3,0.5,2.2,6.8,0.70
1,1209.0,23.0,0.0,12.0,22.0,2.16
2,1965.2,75.5,69.8,15.3,6.8,0.50


In [17]:
t1 = time.perf_counter()
imputer = MissForest(n_jobs=-1, random_state=42)
nutrition_table_filled = imputer.fit_transform(nutrition_table_to_fill)
t2 = time.perf_counter()
print(f"Temps d'exécution : {round(t2-t1)//60} minutes.")

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Temps d'exécution : 20 minutes.


In [18]:
nutrition_table_filled = pd.DataFrame(data=nutrition_table_filled, columns=nutrition_table_to_fill.columns)
nutrition_table_filled.head(3)

,reconstructed_energy_100g,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,salt_100g
0,457.1,15.3,0.5,2.2,6.8,0.70
1,1209.0,23.0,0.0,12.0,22.0,2.16
2,1965.2,75.5,69.8,15.3,6.8,0.50


In [19]:
get_missing_values_table(nutrition_table_filled)

>>> df contient 0 colonnes avec des valeurs manquantes.


""
Missing Values
% of Total Values


In [20]:
other_attribs = ["product_name", "countries_en", "packaging", "nutriscore_grade",
                 "nutriscore_score", "additives_n", "other_carbs", "g_sum"]

other_nutrition_table = nutrition_table[other_attribs]
other_nutrition_table.head(3)

,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,other_carbs,g_sum
0,blanquette de volaille et son riz,france,"carton,plastique",B,0.0,2.0,14.8,25.00
1,salade cesar,canada,frais,C,6.0,3.0,23.0,59.16
2,entremets crème brulée,france,sachet,B,2.0,4.0,5.7,98.10


Concatenons la table correspondant aux macronutriments `nutrition_table_filled` avec la table `other_nutrition_table` contenant les autres variables :

In [21]:
nutrition_table_complete = pd.concat([other_nutrition_table, nutrition_table_filled], axis=1)
nutrition_table_complete.head(3)

,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,other_carbs,g_sum,reconstructed_energy_100g,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,salt_100g
0,blanquette de volaille et son riz,france,"carton,plastique",B,0.0,2.0,14.8,25.00,457.1,15.3,0.5,2.2,6.8,0.70
1,salade cesar,canada,frais,C,6.0,3.0,23.0,59.16,1209.0,23.0,0.0,12.0,22.0,2.16
2,entremets crème brulée,france,sachet,B,2.0,4.0,5.7,98.10,1965.2,75.5,69.8,15.3,6.8,0.50


In [22]:
get_missing_values_table(nutrition_table_complete)

>>> df contient 2 colonnes avec des valeurs manquantes.


,g_sum,other_carbs
Missing Values,887.00,872.0
% of Total Values,0.51,0.5


Calculons les variables `g_sum` et `other_carbs` maintenant que tous les autres champs nécessaires à leurs calculs ont été remplis : 

In [23]:
nutrition_table_complete['g_sum'] = nutrition_table_complete['g_sum'].fillna(
                                    nutrition_table_complete['fat_100g'] +\
                                    nutrition_table_complete['carbohydrates_100g'] +\
                                    nutrition_table_complete['proteins_100g'] +\
                                    nutrition_table_complete['salt_100g']) 

nutrition_table_complete['other_carbs'] = nutrition_table_complete['other_carbs'].fillna(
                                          nutrition_table_complete['carbohydrates_100g'] -\
                                          nutrition_table_complete['sugars_100g'])

In [24]:
get_missing_values_table(nutrition_table_complete)

>>> df contient 0 colonnes avec des valeurs manquantes.


""
Missing Values
% of Total Values


`MissForest()` a prédit des valeurs négatives pour la variables `other_carbs`, supprimons les observations associées :

In [25]:
positive_other_carbs = nutrition_table_complete["other_carbs"] > 0
nutrition_table_complete = nutrition_table_complete[positive_other_carbs].reset_index(drop=True)
nutrition_table_complete.head(3)

,product_name,countries_en,packaging,nutriscore_grade,nutriscore_score,additives_n,other_carbs,g_sum,reconstructed_energy_100g,carbohydrates_100g,sugars_100g,fat_100g,proteins_100g,salt_100g
0,blanquette de volaille et son riz,france,"carton,plastique",B,0.0,2.0,14.8,25.00,457.1,15.3,0.5,2.2,6.8,0.70
1,salade cesar,canada,frais,C,6.0,3.0,23.0,59.16,1209.0,23.0,0.0,12.0,22.0,2.16
2,entremets crème brulée,france,sachet,B,2.0,4.0,5.7,98.10,1965.2,75.5,69.8,15.3,6.8,0.50


Sauvegarde de notre table nettoyée au format `.csv` : 

In [26]:
nutrition_table_complete.to_csv("nutrition_table.csv")